In [ ]:
from label_studio_sdk.client import LabelStudio
from label_studio_sdk.data_manager import Filters, Operator
import json

In [ ]:
LS_URL = "https://ls.kpi.kul.pl/"
API_TOKEN = "e576444cc03ec6e7abf9d86f441523f1b7a6efb3"
PROJECT_ID = 1  # <- replace
CHUNK_SIZE = 200  # keep the request small

client = LabelStudio(base_url=LS_URL, api_key=API_TOKEN)
project = client.projects.get(PROJECT_ID)

In [ ]:
import label_studio_sdk

label_studio_sdk.__version__

In [ ]:
from label_studio_sdk import Client
from label_studio_sdk.data_manager import Type

ls = Client(url=LS_URL, api_key=API_TOKEN)
project = ls.get_project(PROJECT_ID)

# filters = Filters.create(
#     Filters.AND,
#     [
#         Filters.item(
#             # works only on LS 1.11+
#             Column.data("id"),          # or just the string 'tasks:storage_filename'
#             Operator.CONTAINS,                # EQUAL / REGEX / etc. also work
#             Type.String,
#             Filters.value("2024/05/my_image.jpg")
#         )
#     ]
# )

filters = Filters.create(
    Filters.AND,
    [
        Filters.item(
            "tasks:storage_filename",
            Operator.CONTAINS,
            Type.String,
            Filters.value("chelmno_1871"),
        )
    ],
)

tasks = project.get_tasks(filters=filters)
print(tasks)

In [ ]:
def visualize_tokens_with_labels(image_path, words, bboxes, labels, output_path=None):
    """
    Visualize tokens with bounding boxes and colored labels on the image.
    Ignores tokens with label "O".
    Available labels: {'O', 'building_material', 'dedication', 'parish'}
    """
    from PIL import Image, ImageDraw, ImageFont

    # Define colors for each label
    label2color = {
        "building_material": "red",
        "dedication": "orange",
        "parish": "blue",
        "deanery": "green",
    }

    # Open image
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        draw = ImageDraw.Draw(img)
        try:
            font = ImageFont.truetype("Arial", 14)
        except IOError:
            font = ImageFont.load_default()

        for word, bbox, label in zip(words, bboxes, labels):
            if label == "O":
                continue
            color = label2color.get(label, "black")
            draw.rectangle(bbox, outline=color, width=2)
            # Draw label and word above the box
            label_text = f"{label}: {word}"
            # Use textbbox instead of textsize
            left, top, right, bottom = draw.textbbox((0, 0), label_text, font=font)
            text_width = right - left
            text_height = bottom - top
            text_x, text_y = bbox[0], max(0, bbox[1] - text_height)
            draw.rectangle(
                [text_x, text_y, text_x + text_width, text_y + text_height],
                fill=(255, 255, 255, 180),
            )
            draw.text((text_x, text_y), label_text, fill=color, font=font)

        if output_path:
            img.save(output_path)
        return img

In [ ]:
tasks_dict = {task["storage_filename"].split("/")[-1]: task for task in tasks}
tasks_dict

In [ ]:
import numpy as np


def merge_bio_entities(bboxes, labels, tokens, verbose=False):
    """
    Merge consecutive entity tokens into single bounding boxes and tokens,
    avoiding merging across apparent line breaks or large gaps.

    Args:
        bboxes (list): List of bounding boxes [x1, y1, x2, y2].
        labels (list): List of label strings (e.g., "parish", "deanery", "O").
        tokens (list): List of tokens/words.
        verbose (bool): If True, print debug info.

    Returns:
        merged_boxes (list): List of merged bounding boxes.
        merged_tokens (list): List of merged entity strings.
        merged_classes (list): List of merged entity class names (e.g., 'parish').
    """
    # Validate inputs
    if not (len(bboxes) == len(labels) == len(tokens)):
        raise ValueError("bboxes, labels, and tokens must have the same length")

    merged_boxes = []
    merged_tokens = []
    merged_classes = []

    bbox_stack = []
    token_stack = []
    current_entity_type = None

    def merge_entity(stack_bboxes, stack_tokens, entity_type):
        """Merge stacked boxes and tokens into a single entity."""
        valid_bboxes_np = np.array(stack_bboxes)
        merged_bbox = [
            np.min(valid_bboxes_np[:, 0]),  # min x1
            np.min(valid_bboxes_np[:, 1]),  # min y1
            np.max(valid_bboxes_np[:, 2]),  # max x2
            np.max(valid_bboxes_np[:, 3]),  # max y2
        ]
        merged_token = " ".join(stack_tokens)
        return merged_bbox, merged_token, entity_type

    for i, (bbox, label, token) in enumerate(zip(bboxes, labels, tokens)):
        # Case 1: Current token is "O" (outside)
        if label == "O":
            if bbox_stack:  # Finalize the previous entity
                merged_bbox, merged_token, entity_class = merge_entity(
                    bbox_stack, token_stack, current_entity_type
                )
                if merged_bbox:
                    merged_boxes.append(merged_bbox)
                    merged_tokens.append(merged_token)
                    merged_classes.append(entity_class)
                    if verbose:
                        print(
                            f"Finalized entity (due to O): {entity_class} -> '{merged_token}'"
                        )
                bbox_stack = []
                token_stack = []
                current_entity_type = None
            # Do nothing else for "O"

        # Case 2: Current token has a label (entity)
        else:
            # If this is a new entity or different from the current one
            if not bbox_stack or current_entity_type != label:
                # Finalize any previous entity first
                if bbox_stack:
                    merged_bbox, merged_token, entity_class = merge_entity(
                        bbox_stack, token_stack, current_entity_type
                    )
                    if merged_bbox:
                        merged_boxes.append(merged_bbox)
                        merged_tokens.append(merged_token)
                        merged_classes.append(entity_class)
                        if verbose:
                            print(
                                f"Finalized entity (new entity): {entity_class} -> '{merged_token}'"
                            )

                # Start new entity
                bbox_stack = [bbox]
                token_stack = [str(token)]  # Ensure token is string
                current_entity_type = label
                if verbose:
                    print(f"Starting new entity: {current_entity_type}, token: {token}")

            # Continuing the same entity type
            else:
                # Check for line break or large gap before continuing
                last_bbox = bbox_stack[-1]
                current_bbox = bbox

                # Heuristic thresholds (adjust as needed)
                max_vertical_dist_factor = (
                    0.7  # Allow center diff up to 70% of last box height
                )
                max_horizontal_gap_factor = (
                    2.0  # Allow gap up to 2.0x width of last box
                )

                last_height = last_bbox[3] - last_bbox[1]
                last_width = last_bbox[2] - last_bbox[0]
                last_center_y = (last_bbox[1] + last_bbox[3]) / 2
                current_center_y = (current_bbox[1] + current_bbox[3]) / 2

                vertical_dist = abs(current_center_y - last_center_y)
                horizontal_gap = (
                    current_bbox[0] - last_bbox[2]
                )  # Positive -> gap, Negative -> overlap

                is_break = False
                # 1. Check for significant vertical distance (likely new line)
                if (
                    last_height > 1
                    and vertical_dist > last_height * max_vertical_dist_factor
                ):
                    is_break = True
                    if verbose:
                        print(
                            f"Break detected (Vertical): Entity {current_entity_type}, Token '{token}', VDist: {vertical_dist:.1f} > Threshold: {last_height * max_vertical_dist_factor:.1f}"
                        )
                # 2. Check for large horizontal gap (only if vertically close)
                elif (
                    last_width > 1
                    and horizontal_gap > last_width * max_horizontal_gap_factor
                    and vertical_dist <= last_height * max_vertical_dist_factor
                ):
                    is_break = True
                    if verbose:
                        print(
                            f"Break detected (Horizontal Gap): Entity {current_entity_type}, Token '{token}', HGap: {horizontal_gap:.1f} > Threshold: {last_width * max_horizontal_gap_factor:.1f}"
                        )
                # 3. Check for wrap-around (significant move left AND down)
                elif (
                    last_width > 1
                    and current_bbox[0] < (last_bbox[0] - last_width * 0.5)
                    and current_center_y > last_center_y + last_height * 0.1
                ):
                    is_break = True
                    if verbose:
                        print(
                            f"Break detected (Wrap Around): Entity {current_entity_type}, Token '{token}', X1_curr: {current_bbox[0]:.1f} < Threshold: {last_bbox[0] - last_width * 0.5:.1f}"
                        )

                if not is_break:
                    # Continue the current entity
                    bbox_stack.append(current_bbox)
                    token_stack.append(str(token))
                    if verbose:
                        print(
                            f"Continuing entity: {current_entity_type}, token: {token}"
                        )
                else:
                    # Finalize the previous entity due to break
                    merged_bbox, merged_token, entity_class = merge_entity(
                        bbox_stack, token_stack, current_entity_type
                    )
                    if merged_bbox:
                        merged_boxes.append(merged_bbox)
                        merged_tokens.append(merged_token)
                        merged_classes.append(entity_class)
                        if verbose:
                            print(
                                f"Finalized entity (due to break): {entity_class} -> '{merged_token}'"
                            )

                    # Start new entity with the current token
                    bbox_stack = [current_bbox]
                    token_stack = [str(token)]
                    # current_entity_type remains the same (as we're still in the same entity type)
                    if verbose:
                        print(
                            f"Starting new entity (after break): {current_entity_type}, token: {token}"
                        )

    # Final check: Merge any remaining entity after the loop
    if bbox_stack:
        merged_bbox, merged_token, entity_class = merge_entity(
            bbox_stack, token_stack, current_entity_type
        )
        if merged_bbox:
            merged_boxes.append(merged_bbox)
            merged_tokens.append(merged_token)
            merged_classes.append(entity_class)
            if verbose:
                print(
                    f"Finalized entity (end of list): {entity_class} -> '{merged_token}'"
                )

    # Final validation for visualization function compatibility
    if not (len(merged_boxes) == len(merged_tokens) == len(merged_classes)):
        print("Error: Length mismatch after merging! Check verbose logs.")
        print(
            f"Boxes: {len(merged_boxes)}, Tokens: {len(merged_tokens)}, Classes: {len(merged_classes)}"
        )

    return merged_boxes, merged_tokens, merged_classes

In [ ]:
from typing import Tuple


def pixel_bbox_to_percent(
    bbox: Tuple[int, int, int, int], image_width: int, image_height: int
) -> Tuple[float, float, float, float]:
    """
    Zamienia bbox w pikselach (x1,y1,x2,y2)
    na wartości procentowe (x%, y%, width%, height%).
    """
    x1, y1, x2, y2 = bbox
    x_pct: float = (x1 / image_width) * 100.0
    y_pct: float = (y1 / image_height) * 100.0
    width_pct: float = ((x2 - x1) / image_width) * 100.0
    height_pct: float = ((y2 - y1) / image_height) * 100.0
    return x_pct, y_pct, width_pct, height_pct

In [ ]:
import os
from PIL import Image

ROOT_DIR = "/Users/user/Projects/AI_Osrodek/"
LLM_ANNOTATIONS_DIR = "data/llm_annotations"
SCHEMATISM_DIR = "data/schematyzmy"
schematims_to_synchronize = [
    schematism
    for schematism in os.listdir(os.path.join(ROOT_DIR, LLM_ANNOTATIONS_DIR))
    if not schematism.startswith(".")
    and os.path.isdir(os.path.join(ROOT_DIR, LLM_ANNOTATIONS_DIR, schematism))
]

for schematism in schematims_to_synchronize:
    print("Processing schematism:", schematism)
    filters = Filters.create(
        Filters.AND,
        [
            Filters.item(
                "tasks:storage_filename",
                Operator.CONTAINS,
                Type.String,
                Filters.value(schematism),
            )
        ],
    )

    tasks = project.get_tasks(filters=filters)
    tasks_dict = {task["storage_filename"].split("/")[-1]: task for task in tasks}

    for annotation_json in os.listdir(
        os.path.join(ROOT_DIR, LLM_ANNOTATIONS_DIR, schematism)
    ):
        print("Processing annotation:", annotation_json)

        if not annotation_json.endswith(".json"):
            continue

        with open(
            os.path.join(ROOT_DIR, LLM_ANNOTATIONS_DIR, schematism, annotation_json),
            "r",
        ) as f:
            data = json.load(f)
            words, bboxes, labels = data["words"], data["bboxes"], data["labels"]

        # Get the corresponding image filename
        image_filename = annotation_json.replace(".json", ".jpg")

        # Check if we have this image in our tasks
        if image_filename not in tasks_dict:
            print(f"Warning: No task found for {image_filename}")
            continue

        # Get the specific task for this annotation
        task = tasks_dict[image_filename]
        image_path = os.path.join(ROOT_DIR, SCHEMATISM_DIR, schematism, image_filename)

        try:
            image = Image.open(image_path)
            image_width, image_height = image.size
            image = image.convert("RGB")

            merged_bboxes, merged_sentences, merged_classes = merge_bio_entities(
                bboxes,
                labels,
                words,
                verbose=False,
            )

            annotated_image = visualize_tokens_with_labels(
                image_path=image_path,
                words=merged_sentences,
                bboxes=merged_bboxes,
                labels=merged_classes,
                # output_path=os.path.join(ROOT_DIR, SCHEMATISM_DIR, schematism, image_filename.replace(".jpg", "_visualized.jpg")),
            )

            display(annotated_image)

            # Create results for this specific task
            current_task_results = []
            for bbox, class_name in zip(merged_bboxes, merged_classes):
                x_percent, y_percent, width_percent, height_percent = (
                    pixel_bbox_to_percent(
                        bbox=bbox,
                        image_width=image_width,
                        image_height=image_height,
                    )
                )
                result_item = {
                    "type": "rectanglelabels",
                    "from_name": "label",  # Make sure this matches your Label Studio config_manager
                    "to_name": "image",  # Make sure this matches your Label Studio config_manager
                    "original_width": image_width,
                    "original_height": image_height,
                    "image_rotation": 0,
                    "value": {
                        "rectanglelabels": [class_name],
                        "x": x_percent,
                        "y": y_percent,
                        "width": width_percent,
                        "height": height_percent,
                    },
                    "score": 1.0,
                }
                current_task_results.append(result_item)

            # Add a single prediction for this task
            prediction = {
                "task": task["id"],
                "result": current_task_results,
                "score": 0.9,  # Overall confidence score
                "model_version": "llm_mistral_0.0.1",
            }

            # Upload prediction for this specific task
            upload_response = project.create_predictions(
                predictions=[prediction],
            )
            print(f"Upload response for task {task['id']}: {upload_response}")

        except Exception as e:
            print(f"Error processing {image_filename}: {str(e)}")